# Parse

In [1]:

import os
import re
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import csv
import time
import datetime
import functools

In [2]:
directory = '/Users/marcelo.gimenes/Desktop/habits/9883a7bd-d679-4ced-9668-60cccaf0db74_Export-a20a60bd-e5e9-4997-aba2-9cd867f832f6/New Habit Tracker b486659186084d25b3495f08b2418fdc/'

In [3]:
def remove_last_word(phrase):
    return ' '.join(phrase.split(' ')[:-1])

def get_csv_in_folder(folder):
    csvs = [ filename for filename in os.listdir(folder) if filename.endswith(".csv") ]
    return f"{folder}/{csvs[0]}"

In [4]:
csvs = [ {"path": get_csv_in_folder(f.path), "name": remove_last_word(f.name)} for f in os.scandir(directory) if f.is_dir() ]
csvs

[{'path': '/Users/marcelo.gimenes/Desktop/habits/9883a7bd-d679-4ced-9668-60cccaf0db74_Export-a20a60bd-e5e9-4997-aba2-9cd867f832f6/New Habit Tracker b486659186084d25b3495f08b2418fdc/Cycle for Weeks 13 and 14 1cf1492b5b8d4997be93f7af35adb2d1/Habits 9abb9cbda8e846c182c59fbad273d098.csv',
  'name': 'Cycle for Weeks 13 and 14'},
 {'path': '/Users/marcelo.gimenes/Desktop/habits/9883a7bd-d679-4ced-9668-60cccaf0db74_Export-a20a60bd-e5e9-4997-aba2-9cd867f832f6/New Habit Tracker b486659186084d25b3495f08b2418fdc/Cycle for Weeks 19 and 20 5cb6888d0f55435fb0b9f182ff1662fa/Habits 32606a61bef241d5a4b51c5ed88ea7b2.csv',
  'name': 'Cycle for Weeks 19 and 20'},
 {'path': '/Users/marcelo.gimenes/Desktop/habits/9883a7bd-d679-4ced-9668-60cccaf0db74_Export-a20a60bd-e5e9-4997-aba2-9cd867f832f6/New Habit Tracker b486659186084d25b3495f08b2418fdc/Cycle for Weeks 7 and 8 7a37acc1867d4511843f5a514d897a5a/Habits b37521a68ee3490d963bd13ee2e42649.csv',
  'name': 'Cycle for Weeks 7 and 8'},
 {'path': '/Users/marcelo.

In [5]:
def read_csv(filename):
    with open(filename) as csvfile:
        reader = csv.reader(csvfile)
        return list(reader)

In [6]:
def date_from_weekno(year, week, weekday):
    return datetime.datetime.fromtimestamp(time.mktime(time.strptime(f"{year} {week} {weekday}", '%Y %U %w')))

In [7]:
def df_from_cycle(cycle):
    name = cycle["name"]
    data = np.array(cycle["data"][1:])
    data_T = data.T
    df = pd.DataFrame(data_T[1:], columns=data_T[0])
    weeks = ' '.join(name.split(' ')[-3:]).split(' and ')
    df["weekday"] = df.index % 7
    df["week"] = np.where(df.index > 6, int(weeks[1]), int(weeks[0]))
    df["date"] = df.apply(lambda x: date_from_weekno(2023, x.week, x.weekday), axis=1)
    return df.set_index("date")

In [8]:
cycles = map(lambda x: {"data": read_csv(x["path"]), "name": x["name"]}, csvs)
cycles = filter(lambda x: "N and M" not in x["name"], cycles)

dfs = map(df_from_cycle, cycles)
df = functools.reduce(lambda a, b: pd.concat([a, b]), dfs, pd.DataFrame())
df.sort_index(inplace=True)
df.drop(["2023-06-16", "2023-06-17"], inplace=True)
df

,Wake up at 8h30,Take some vitamins,Pray,Read the Bible (Psalms),Read (spiritual),Study (work),Read (philosophy),Chores,Work,Drink water (2.8L),...,Wake up at 7h30,Remove one video from YT's WL (885),Use less glasses,Read the Bible (Psalm 119),Remove one video from YT's WL (912),Study (audio),Don't play games,Remove 3 songs from LS and 1 playlist,Start eating breakfast before 9am,Stretch
date,,,,,,,,,,,,,,,,,,,,,
2023-01-01,NaN,NP NOK,NP NOK,NaN,NaN,NaN,NaN,NaN,NP NOK,NaN,...,NP NOK,NaN,NP NOK,NP NOK,NP NOK,NaN,NaN,NaN,NaN,NaN
2023-01-02,NaN,OK,NOK,NaN,NaN,NaN,NaN,NaN,NP NOK,NaN,...,OK,NaN,NP NOK,NOK,NP NOK,NaN,NaN,NaN,NaN,NaN
2023-01-03,NaN,OK,OK,NaN,NaN,NaN,NaN,NaN,OK,NaN,...,OK,NaN,NOK,OK,OK,NaN,NaN,NaN,NaN,NaN
2023-01-04,NaN,OK,OK,NaN,NaN,NaN,NaN,NaN,OK,NaN,...,NOK,NaN,NOK,OK,OK,NaN,NaN,NaN,NaN,NaN
2023-01-05,NaN,OK,OK,NaN,NaN,NaN,NaN,NaN,OK,NaN,...,NOK,NaN,NOK,OK,OK,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-11,NaN,NOK,OK,OK,OK,OK,OK,OK,NP NOK,NOK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OK,NOK
2023-06-12,NaN,OK,NOK,NOK,OK,OK,NOK,NOK,OK,NOK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OK,NOK
2023-06-13,NaN,NP NOK,OK,OK,OK,OK,OK,OK,NP NOK,OK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OK,OK


# Generate Markdowns

In [9]:
for index, row in df.iterrows():
    clean_row = row.drop(labels=["weekday", "week"]).dropna()

    markdown = "# Habits\n"
    for label, value in clean_row.items():
        if value == "NP NOK":
            continue
        elif value == "NOK":
            markdown += f"- [ ] {label}\n"
        elif value == "OK":
            markdown += f"- [x] {label}\n"
        else:
            raise Exception(f"Error ({index}) - {label}: '{value}'")


    file = open(f"output/{index.date()}.md", "w")
    file.write(markdown)
    file.close()

# Plot

In [10]:
# df.drop(columns=[
#     "Remove one video from YT's WL (885)", 
#     "Remove one video from YT's WL (912)",
#     'Sync Invoices database',
#     'Remove 3 songs from LS',
#     'Remove 3 songs from LS and 1 playlist',
#     'Study (audio)'
# ], inplace=True)

In [11]:
# columns = df.columns.to_numpy()
# columns.sort()
# columns